# Decision Trees

We'll be looking at individual income in the United States. The data is from the 1994 census, and contains information on an individual's marital status, age, type of work, and more. The target column, or what we want to predict, is whether individuals make less than or equal to 50k a year, or more than 50k a year.

In [2]:
import pandas

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)
print(income.head(5))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

In [3]:
income.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education_num      int64
marital_status    object
occupation        object
relationship      object
race              object
sex               object
capital_gain       int64
capital_loss       int64
hours_per_week     int64
native_country    object
high_income       object
dtype: object

In [4]:
def convert_categorical_vars(df):
    text_cols = df.select_dtypes(include=['object']).columns
    for col in text_cols:
        cat_col = pandas.Categorical(df[col])
        df[col] = cat_col.codes

In [ ]:
# Convert a single column from text categories to numbers
#col = pandas.Categorical(income["workclass"])
#income["workclass"] = col.codes
#print(income["workclass"].head(5))

In [5]:
#Convert all columns
print(income.head(5))
convert_categorical_vars(income)
print(income.head(5))

   age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country high_income  
0          2174             0              40   United-States   

* Now think of rows of data flowing through a decision tree. 
* Next, we'll split the data set into two portions based on whether the individual works in the private sector or not.

In [6]:
private_incomes = income[income['workclass'] == 4]
print("Total private incomes: "+str(len(private_incomes)))
public_incomes = income[income['workclass'] != 4]
print("Total public incomes: "+str(len(public_incomes)))

Total private incomes: 22696
Total public incomes: 9865


In [19]:
print(list(income['high_income'].value_counts().index))

[0, 1]


-\sum_{i=1}^{c} {\mathrm{P}(x_i) \log_b \mathrm{P}(x_i)}
## Entropy
SUM(i=1 to c)[P(x_i)*log_b(P(x_i))]

* **Entropy** is a measure for how "mixed together" the values for a variable are. If a variable has only one possible value, that is low entropy.

We iterate through each unique value in a single column and assign it to i. We then compute the probability of that value occurring in the data (P(x_i)). Next we do some multiplication, and sum all of the values together. b is the base of the logarithm. We commonly use the value 2 for this, but we can also set it to 10 or another value.

## Information Gain
IG(T,A) = Entropy(T) - SUM(v element of A)[(|T_v|/|T|)*Entropy(T_v)]

Compute the entropy of the high_income column in the income dataframe, and assign the result to income_entropy.

In [20]:
import math

def compute_entropy(df,column):
    col_unique_vals = list(df[column].value_counts().index)
    total_rows = len(df)
    entropy = 0
    for i in col_unique_vals:
        value_instances = len(df[df[column] == i])
        entropy += (value_instances/total_rows) * math.log((value_instances/total_rows), 2)
    return -entropy

## Passing in 2 as the second parameter to math.log will take a base 2 log
#income_entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
income_entropy = compute_entropy(income,'high_income')
print(income_entropy)

0.7963839552022132


# Tree Construction Strategies

* **Create branches for each value** - to create as many branches at each node as there are unique values for the variable we're splitting on. So if the variable has three or four values, we'd end up with three or four branches. This approach usually involves more complexity than it's worth and doesn't improve prediction accuracy, but it's worth knowing about.
* **Calculate the average and two branches for above and below** - find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.

Compute the information gain for splitting on the age column of income.

First, compute the median of age.
Then, assign anything less than or equal to the median to the left branch, and anything greater than the median to the right branch.
Compute the information gain and assign it to age_information_gain.

In [ ]:
import numpy

def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

median_age = income['age'].median()
left_branch = income[income['age']<=median_age]
right_branch = income[income['age']>median_age]
age_information_gain = calc_entropy(income['high_income']) - (len(left_branch)/len(income)*calc_entropy(left_branch['high_income']) + len(right_branch)/len(income)*calc_entropy(right_branch['high_income']))

In [ ]:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

information_gains = []

for col in columns:
    information_gains.append(calc_information_gain(income, col, "high_income"))
    
highest_gain = columns[information_gains.index(max(information_gains))]
print("Highest gain from split: "+ highest_gain)

## The ID3 Algorithm for constructing decision trees

```
def id3(data, target, columns)
    1 Create a node for the tree
    2 If all values of the target attribute are 1, Return the node, with label = 1
    3 If all values of the target attribute are 0, Return the node, with label = 0
    4 Using information gain, find A, the column that splits the data best
    5 Find the median value in column A
    6 Split column A into values below or equal to the median (0), and values above the median (1)
    7 For each possible value (0 or 1), vi, of A,
    8    Add a new tree branch below Root that corresponds to rows of data where A = vi
    9    Let Examples(vi) be the subset of examples that have the value vi for A
   10    Below this new branch add the subtree id3(data[A==vi], target, columns)
   11 Return Root
```

In [ ]:
def find_best_column(data, target_name, columns):
    # Fill in the logic here to automatically find the column in columns to split on
    # data is a dataframe
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    information_gains = []
    for col in columns:
        information_gains.append(calc_information_gain(data, col, 'high_income'))
    highest_gain = columns[information_gains.index(max(information_gains))]
    return highest_gain

# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]
income_split = find_best_column(income, 'high_income', columns)

In [23]:
income['high_income'][20]

1

In [ ]:
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1)
    tree["number"] = nodes[-1]

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if unique_targets[0] == 1:
            tree["label"] = 1
        else:
            tree["label"] = 0
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree["column"] = best_column
    tree["median"] = column_median
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)

In [ ]:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    # Don't forget to increment depth when you pass it in
    for branch in branches:
        print_node(branch, depth+1)

print_node(tree, 0)

In [ ]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    # Insert code here to check whether row[column] is less than or equal to median
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    # If it's greater, return the result of predicting on the right branch of the tree
    # Remember to use the return statement to return the result!
    if row[column] < median:
        return predict(tree["left"],row)
    else:
        return predict(tree["right"], row)

# Print the prediction for the first row in our data
predict_result = predict(tree, data.iloc[0])
print(predict_result)

In [ ]:
new_data = pandas.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    return df.apply(lambda x: predict(tree, x), axis=1)
    #pass

predictions = batch_predict(tree, new_data)